In [1]:
#installation du package texblob en français
!pip install textblob_fr

     |████████████████████████████████| 561 kB 4.9 MB/s 


In [2]:
import pandas as pd 
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
import plotly.graph_objects as go
import plotly.express as px

In [3]:
data  = pd.read_csv('/content/drive/MyDrive/Projet_s1/wikiconflit/df_eoli.csv')

In [4]:
data.columns

Index(['Unnamed: 0', 'Thématique', 'contenu_poste'], dtype='object')

In [5]:
data= data.drop(['Unnamed: 0'],axis=1)
data

,Thématique,contenu_poste
0,éolienne,Question : comment se fait-il que ce commentai...
1,éolienne,Ce message ? Il a été déplacé plus bas et pas ...
2,éolienne,N.B. : le message dans lequel s'insérait ce me...
3,éolienne,Il existe 2 solutions : soit tu modifie manuel...
4,éolienne,"Merci, je cherchais moi aussi les références d..."
...,...,...
230,éolienne,"Bonjour Televore, WP:N'hésitez pas à faire cet..."
231,éolienne,"Bonjour, Malheureusement cet article est en Se..."
232,éolienne,"Bonjour, Malheureusement cet article est en Se..."
233,éolienne,"Bonjour Televore, j'ai complété par une référe..."


In [ ]:
#Affichons les valeurs négatives (nulles)
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["contenu_poste", "percentage"])
NAN.sort_values("percentage", ascending=False)

,contenu_poste,percentage
0,Thématique,0.0
1,contenu_poste,0.0


In [ ]:
# converion des données en miniscule
data['contenu_poste'].str.lower()

0      question : comment se fait-il que ce commentai...
1      ce message ? il a été déplacé plus bas et pas ...
2      n.b. : le message dans lequel s'insérait ce me...
3      il existe 2 solutions : soit tu modifie manuel...
4      merci, je cherchais moi aussi les références d...
                             ...                        
230    bonjour televore, wp:n'hésitez pas à faire cet...
231    bonjour, malheureusement cet article est en se...
232    bonjour, malheureusement cet article est en se...
233    bonjour televore, j'ai complété par une référe...
234    super! merci beaucoup!!!  televore (discuter) ...
Name: contenu_poste, Length: 235, dtype: object

In [ ]:
#Tokenisation des mots et suppression de la ponctuation
discussion = []
for poste in data["contenu_poste"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",poste ).split():
        Word_Tok.append(word)
    discussion.append(Word_Tok)


In [ ]:
discussion

[['Question',
  'comment',
  'se',
  'fait',
  'il',
  'que',
  'ce',
  'commentaire',
  'soit',
  'supprimé',
  'actu',
  'diff',
  '30',
  'juillet',
  '2007',
  'à',
  '21',
  '53',
  '195',
  '132',
  '49',
  '80',
  'Discuter',
  '17',
  '720',
  'octets',
  'Unités',
  'défaire',
  'alors',
  'que',
  'tout',
  'à',
  'fait',
  'juste',
  'Le',
  'message',
  'qui',
  'précède',
  'non',
  'signé',
  'a',
  'été',
  'déposé',
  'par',
  'IP',
  '129',
  '184',
  '84',
  '10',
  'discuter',
  'le',
  '31',
  'juillet',
  '2007',
  'à',
  '17',
  '21',
  'Ce',
  'message',
  'Il',
  'a',
  'été',
  'déplacé',
  'plus',
  'bas',
  'et',
  'pas',
  'supprimé',
  'les',
  'commentaires',
  'les',
  'plus',
  'récents',
  'vont',
  'en',
  'bas',
  'de',
  'ceux',
  'déjà',
  'mis',
  'Et',
  'dans',
  'le',
  'cas',
  'de',
  'ce',
  'message',
  'en',
  'particulier',
  'il',
  's',
  'agissait',
  'd',
  'un',
  'nouveau',
  'sujet',
  'de',
  'discussion',
  'et',
  'pas',
  'd',
 

In [ ]:
#ajoutons une nouvelle colonne celle des mots tokénisés à notre dataset
data['mot_tokenisé'] = discussion
data.head()

,Thématique,contenu_poste,mot_tokenisé,posteAferPreproc
0,éolienne,Question : comment se fait-il que ce commentai...,"[Question, comment, se, fait, il, que, ce, com...",Question commentaire supprimé actu diff 30 jui...
1,éolienne,Ce message ? Il a été déplacé plus bas et pas ...,"[Ce, message, Il, a, été, déplacé, plus, bas, ...",Ce message Il déplacé plus pas supprimé commen...
2,éolienne,N.B. : le message dans lequel s'insérait ce me...,"[N, B, le, message, dans, lequel, s, insérait,...",message insérait message avez modifié commenta...
3,éolienne,Il existe 2 solutions : soit tu modifie manuel...,"[Il, existe, 2, solutions, soit, tu, modifie, ...",Il existe solutions modifie manuellement artic...
4,éolienne,"Merci, je cherchais moi aussi les références d...","[Merci, je, cherchais, moi, aussi, les, référe...",Merci cherchais références stateolien 4e docum...


In [ ]:
stop_words=set(STOP_WORDS)
deselect_stop_words = ['n\'', 'ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

In [ ]:
Allfilteredposte = []
for poste in data["mot_tokenisé"]:
    filteredposte = [w for w in poste if not ((w in stop_words) or (len(w) == 1))]
    Allfilteredposte.append(' '.join(filteredposte))

In [ ]:
# Ajout d'une nouvelle colonne après le processing
data["CommentAferPreproc"]=Allfilteredposte
data.head()

,Thématique,contenu_poste,mot_tokenisé,posteAferPreproc,CommentAferPreproc
0,éolienne,Question : comment se fait-il que ce commentai...,"[Question, comment, se, fait, il, que, ce, com...",Question commentaire supprimé actu diff 30 jui...,Question commentaire supprimé actu diff 30 jui...
1,éolienne,Ce message ? Il a été déplacé plus bas et pas ...,"[Ce, message, Il, a, été, déplacé, plus, bas, ...",Ce message Il déplacé plus pas supprimé commen...,Ce message Il déplacé plus pas supprimé commen...
2,éolienne,N.B. : le message dans lequel s'insérait ce me...,"[N, B, le, message, dans, lequel, s, insérait,...",message insérait message avez modifié commenta...,message insérait message avez modifié commenta...
3,éolienne,Il existe 2 solutions : soit tu modifie manuel...,"[Il, existe, 2, solutions, soit, tu, modifie, ...",Il existe solutions modifie manuellement artic...,Il existe solutions modifie manuellement artic...
4,éolienne,"Merci, je cherchais moi aussi les références d...","[Merci, je, cherchais, moi, aussi, les, référe...",Merci cherchais références stateolien 4e docum...,Merci cherchais références stateolien 4e docum...


# Analyse de sentiment avec TextBlob

In [ ]:
senti_list = []
for i in data["posteAferPreproc"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positif')
    elif (vs < 0):
        senti_list.append('Negatif')
    else:
        senti_list.append('Neutre') 

In [ ]:
# Ajoutons une colonne sentiment
data["sentiment"]=senti_list
data.head()

,Thématique,contenu_poste,mot_tokenisé,posteAferPreproc,CommentAferPreproc,sentiment
0,éolienne,Question : comment se fait-il que ce commentai...,"[Question, comment, se, fait, il, que, ce, com...",Question commentaire supprimé actu diff 30 jui...,Question commentaire supprimé actu diff 30 jui...,Negatif
1,éolienne,Ce message ? Il a été déplacé plus bas et pas ...,"[Ce, message, Il, a, été, déplacé, plus, bas, ...",Ce message Il déplacé plus pas supprimé commen...,Ce message Il déplacé plus pas supprimé commen...,Negatif
2,éolienne,N.B. : le message dans lequel s'insérait ce me...,"[N, B, le, message, dans, lequel, s, insérait,...",message insérait message avez modifié commenta...,message insérait message avez modifié commenta...,Neutre
3,éolienne,Il existe 2 solutions : soit tu modifie manuel...,"[Il, existe, 2, solutions, soit, tu, modifie, ...",Il existe solutions modifie manuellement artic...,Il existe solutions modifie manuellement artic...,Positif
4,éolienne,"Merci, je cherchais moi aussi les références d...","[Merci, je, cherchais, moi, aussi, les, référe...",Merci cherchais références stateolien 4e docum...,Merci cherchais références stateolien 4e docum...,Positif


In [ ]:
Number_sentiment= data.groupby(["sentiment"])["contenu_poste"].count().reset_index().reset_index(drop=True)

In [ ]:
fig = px.histogram(data, x="sentiment",color="sentiment")
fig.update_layout(
    title_text='Sentiment discussions éolienne', # title of plot
    xaxis_title_text='Sentiment', # xaxis label
    yaxis_title_text='Compte', # yaxis label
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

On constacte qu'il y a plus de sentiments positif  que negatif et neutre qui ont les mêmes proportion 

In [ ]:
#https://www.kaggle.com/fedi1996/french-sentiment-analysis-using-textblob

In [ ]:
fig = px.pie(Number_sentiment, values=Number_sentiment['contenu_poste'],
             names=Number_sentiment['sentiment'], 
             color_discrete_sequence=px.colors.sequential.Emrld
)
fig.show()

D'après nos analyse il ressort que que environ 58% des discussions on des avis positifs sur les éoliennes cpntre 21% neutre et 21% négatifs. 